In [1]:
from typing import List, Tuple
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import CouldNotRetrieveTranscript
import urllib.request
import re, os
import numpy as np
import pandas as pd
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
local_llm = Ollama(
    model="mistral", 
    request_timeout=60.0
)

local_embed_model = OllamaEmbedding(
    model_name="mxbai-embed-large",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

llm = AzureOpenAI(
    engine = "gpt-35-turbo",
    model = "gpt-35-turbo",
    api_key = os.getenv('AZURE_OPENAI_API_KEY'),  
    api_version = "2024-02-01",
    azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
)

embed_model = AzureOpenAIEmbedding(
    model = "text-embedding-3-large",
    deployment_name = "text-embedding-3-large",
    api_key = os.getenv('AZURE_OPENAI_API_KEY'),
    api_version = "2024-02-01",
    azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
)

Settings.llm = llm
Settings.embed_model = embed_model

In [4]:
def get_youtube_videos(search_keyword: str) -> list:
    html = urllib.request.urlopen("https://www.youtube.com/results?search_query=" + search_keyword.replace(' ','+'))
    video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
    unique_ids = []
    for id in video_ids:
        if len(unique_ids) == 10:
            break

        if id not in unique_ids:
            try:
                YouTubeTranscriptApi.get_transcript(id)
                unique_ids.append(id)
            except CouldNotRetrieveTranscript:
                pass
            
    return ["https://www.youtube.com/watch?v="+i for i in unique_ids]

get_youtube_videos_tool = FunctionTool.from_defaults(
    fn=get_youtube_videos,
    description="Returns list of 10 youtube video links for given search keyword."
)

In [5]:
youtube_search_agent = ReActAgent.from_tools(
    tools=[get_youtube_videos_tool],
    llm=llm,
    verbose=False,
    context="Purpose: The primary role of this agent is to provide youtube video links for given query. Provide full link in answer."
)

In [6]:
response1 = youtube_search_agent.query("Can you give me 10 videos for Samsung Galaxy S24 Ultra Review?")

In [7]:
print(response1.response)

Here are the 10 YouTube video links for the Samsung Galaxy S24 Ultra Review:
1. https://www.youtube.com/watch?v=XaqOejIaFgM
2. https://www.youtube.com/watch?v=z598clQ71JY
3. https://www.youtube.com/watch?v=HvoBci_GC8A
4. https://www.youtube.com/watch?v=K-JGaqfIOmI
5. https://www.youtube.com/watch?v=2CvO99eGMLk
6. https://www.youtube.com/watch?v=mEBbk38ENDA
7. https://www.youtube.com/watch?v=dFpNqg5F2qQ
8. https://www.youtube.com/watch?v=PNybdmok7m8
9. https://www.youtube.com/watch?v=B4zSHZll8Gk
10. https://www.youtube.com/watch?v=IA62EZx_HFE


In [39]:
def summarize_text(text: str) -> str:
    prompt = f"Summarize the following text in a concise way:\n\n{text}"
    
    response = local_llm.chat([
        ChatMessage(role="user",content=prompt)
    ])
    
    return response.message.content

def youtube_links_to_summary(youtube_links: List[str]) -> bool:
    loader = YoutubeTranscriptReader()
    documents = loader.load_data(ytlinks=youtube_links)
    df = pd.DataFrame([{'doc_id':doc.doc_id,'text':summarize_text(doc.text)} for doc in documents])
    df.to_csv("top_10_summaries.csv",index=False, sep="|")
    return True

# def youtube_links_to_summary(youtube_links: List[str]) -> List[str]:
#     loader = YoutubeTranscriptReader()
#     documents = loader.load_data(ytlinks=youtube_links)
#     df = pd.DataFrame([{'doc_id':doc.doc_id,'text':summarize_text(doc.text)} for doc in documents])
#     df.to_csv("top_10_summaries.csv",index=False, sep="|")
#     return df.text.to_list()

summarize_youtube_video_tool = FunctionTool.from_defaults(
    fn=youtube_links_to_summary,
    # description="Returns array of summaries of all youtube links."
    description="Returns True if summaries are stored in file."
)

In [40]:
summarize_youtube_agent = ReActAgent.from_tools(
    tools=[summarize_youtube_video_tool],
    llm=llm,
    verbose=True,
    # context="Purpose: The primary role of this agent is to give summary of each youtube video link given in input."
    context="Purpose: The primary role of this agent is to save summary of each youtube video link in csv file."
)

In [41]:
# response2 = summarize_youtube_agent.query("Save summary for each youtube links given here: "+response1.response)
response2 = summarize_youtube_agent.query("Give summary for each youtube links given here: "+response1.response)

> Running step abb74110-5377-46c3-9aa4-aa20b33b4a30. Step input: Give summary for each youtube links given here: Here are the 10 YouTube video links for the Samsung Galaxy S24 Ultra Review:
1. https://www.youtube.com/watch?v=XaqOejIaFgM
2. https://www.youtube.com/watch?v=z598clQ71JY
3. https://www.youtube.com/watch?v=HvoBci_GC8A
4. https://www.youtube.com/watch?v=K-JGaqfIOmI
5. https://www.youtube.com/watch?v=2CvO99eGMLk
6. https://www.youtube.com/watch?v=mEBbk38ENDA
7. https://www.youtube.com/watch?v=dFpNqg5F2qQ
8. https://www.youtube.com/watch?v=PNybdmok7m8
9. https://www.youtube.com/watch?v=B4zSHZll8Gk
10. https://www.youtube.com/watch?v=IA62EZx_HFE
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: youtube_links_to_summary
Action Input: {'youtube_links': ['https://www.youtube.com/watch?v=XaqOejIaFgM', 'https://www.youtube.com/watch?v=z598clQ71JY', 'https://www.youtube.com/watch?v=HvoBci_GC8A', 'https://www.youtube.com

In [42]:
print(response2.response)

Based on the summaries of the YouTube video links for the Samsung Galaxy S24 Ultra, it seems that the reviewers have highlighted various aspects of the phone, including its features, performance, and drawbacks. The reviews cover details such as the new features of the S24 Ultra, the positive and negative aspects of the phone, comparisons with previous models, and recommendations for potential buyers. Overall, the reviews provide a comprehensive overview of the Samsung Galaxy S24 Ultra, offering insights into its strengths and weaknesses.


In [25]:
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

def get_top_n_similar_videos(top_n: int) -> List[dict]:
    df = pd.read_csv("top_10_summaries.csv",sep="|")
    documents = []
    for t in df.itertuples():
        documents.append(
            Document(
                doc_id=t.doc_id,
                text=t.text,
                metadata={'video_id':t.doc_id},
                embedding=local_embed_model.get_text_embedding(t.text)
            )
        )

    sim_matrix = np.zeros(shape=(10,10))
    for i in range(len(documents)):
        for j in range(i+1,len(documents)):
            sim_matrix[i][j] = cosine_similarity(documents[i].embedding,documents[j].embedding)
    
    flat_sim_matrix = sim_matrix.flatten()
    indices = np.argpartition(flat_sim_matrix, -top_n)[-top_n:]
    indices = np.flip(indices[np.argsort(flat_sim_matrix[indices])])

    most_similar_pairs = []
    for idx in indices:
        max_idx = np.unravel_index(idx,sim_matrix.shape)
        most_similar_pairs.append({
            'video1': "https://www.youtube.com/watch?v="+documents[max_idx[0]].doc_id,
            'video2': "https://www.youtube.com/watch?v="+documents[max_idx[1]].doc_id,
            'similarity': sim_matrix[max_idx[0]][max_idx[1]]
        })
    
    return most_similar_pairs

most_similar_videos_tool = FunctionTool.from_defaults(
    fn=get_top_n_similar_videos,
    description="Returns list of pairs of youtube links with most similarity."
)

In [26]:
similarity_agent = ReActAgent.from_tools(
    tools=[most_similar_videos_tool],
    llm=llm,
    verbose=True,
    context="Purpose: The primary role of this agent is to give n pairs of most similar youtube videos."
)

In [27]:
response3 = similarity_agent.query("Give me 3 pairs of most similar videos.")

> Running step 67300996-2c83-4cb7-aaf5-11a99daf9c74. Step input: Give me 3 pairs of most similar videos.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_top_n_similar_videos
Action Input: {'top_n': 3}
Observation: [{'video1': 'https://www.youtube.com/watch?v=5_yMa3hzCLs', 'video2': 'https://www.youtube.com/watch?v=2CvO99eGMLk', 'similarity': 0.8210053485068398}, {'video1': 'https://www.youtube.com/watch?v=2CvO99eGMLk', 'video2': 'https://www.youtube.com/watch?v=IA62EZx_HFE', 'similarity': 0.8170498501258571}, {'video1': 'https://www.youtube.com/watch?v=K-JGaqfIOmI', 'video2': 'https://www.youtube.com/watch?v=5_yMa3hzCLs', 'similarity': 0.812141434787413}]
> Running step ab119c0a-dd4b-4eb3-b23a-18600e8b1f05. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Here are the 3 pairs of most similar videos:
1. Pair 1: 
   - Video 1: [https://www.youtube.com

In [5]:
import os
from llama_index.core.agent import ReActAgent
from llama_index.llms.azure_openai import AzureOpenAI
from factory import youtube_search_agent_factory, summarize_youtube_agent_factory, similarity_agent_factory

from dotenv import load_dotenv
load_dotenv()

llm = AzureOpenAI(
    engine = "gpt-35-turbo",
    model = "gpt-35-turbo",
    api_key = os.getenv('AZURE_OPENAI_API_KEY'),  
    api_version = "2024-02-01",
    azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
)

orchestrator = ReActAgent.from_tools(
    tools=[youtube_search_agent_factory, summarize_youtube_agent_factory, similarity_agent_factory],
    llm=llm,
    verbose=True,
    context="You are on orchestration agent. Your job is to decide which agent to run based on the current state of the user and what they've asked to do. Agents are identified by tools."
)

In [6]:
orchestrator.query("Find 3 pairs most similar videos for Iphone 16 pro max review.")

> Running step 81bf059a-e606-4883-8656-a3e538d06e03. Step input: Find 3 pairs most similar videos for Iphone 16 pro max review.


AttributeError: 'function' object has no attribute 'metadata'